In [6]:
from googletrans import Translator

In [9]:
translator = Translator()

In [8]:
from ftlangdetect import detect

result = detect(text="Bugün hava çok güzel", low_memory=False)
print(result)

ModuleNotFoundError: No module named 'ftlangdetect'

In [9]:
# Test the code
text = "Hello, world!"
detected_language = language_detection(text)
print(f"Detected language: {detected_language}")

NameError: name 'language_detection' is not defined

In [10]:
from langdetect import detect, detect_langs

def detect_language_with_confidence(text, confidence_threshold=0.5):
    try:
        # Detect the language of the text
        language = detect(text)
        
        # Get the confidence score
        confidence = detect_langs(text)[0].prob
        
        # Check if the confidence is above the threshold
        if confidence >= confidence_threshold:
            return language, confidence
        else:
            return "Undetermined", confidence
    
    except Exception as e:
        # Handle exceptions, e.g., if the text is too short
        return "Error", 0.0

# Example usage
text_to_detect = "Hello, this is an example text."
language, confidence = detect_language_with_confidence(text_to_detect)

print(f"Detected language: {language} with confidence: {confidence}")


Detected language: en with confidence: 0.9999977366622846


In [9]:
detect("Hello World")

'en'